In [6]:
import pandas as pd
import seaborn as sns
import plotly.express as px
from datetime import datetime as dt
from datetime import timedelta

In [7]:
advisors = pd.read_excel('data\\database.xlsx', sheet_name='advisors')
wash_list = pd.read_excel('data\\database.xlsx', sheet_name='wash_list')
countries = pd.read_excel('data\\database.xlsx', sheet_name='countries')
calendar = pd.read_excel('data\\database.xlsx', sheet_name='calendar')

c:\Users\paolo.losi\_personal\Coding\wash_dashboard\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning:

Data Validation extension is not supported and will be removed

c:\Users\paolo.losi\_personal\Coding\wash_dashboard\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning:

Data Validation extension is not supported and will be removed

c:\Users\paolo.losi\_personal\Coding\wash_dashboard\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning:

Data Validation extension is not supported and will be removed



In [8]:
countries.head()

,CIA Name,ISO 3166 alpha3,Continent,ta_focal,ta_support
0,Afghanistan,AFG,Asia,Ciaran,Katherine
1,Bangladesh,BGD,Asia,Ciaran,Katherine
2,Burkina Faso,BFA,Africa,Franck,NaN
3,Burundi,BDI,Africa,Franck,NaN
4,Central African Republic,CAF,Africa,Franck,NaN


In [9]:
cols = ['#4A628A', '#7AB2D3', '#B9E5E8', '#DFF2EB']
hover_data = {
    'ta_focal': False,
    'ISO 3166 alpha3': False,
    'Focal Point': countries.ta_focal,
}

fig = px.choropleth(
    data_frame=countries,
    locations='ISO 3166 alpha3',
    color='ta_focal',
    color_discrete_sequence=cols,
    hover_name='CIA Name',
    hover_data=hover_data,
    labels={'ta_focal': 'Focal Point'}
)

fig.show()

In [10]:
calendar.type.unique()

array(['Leave Full Day', 'Bank Holiday', 'Conference/Workshop',
       'Time In Lieu Full Day', 'Country Support Visit', 'Training',
       'Time In Lieu Half Day'], dtype=object)

In [11]:
def date_prettify(series):
    return f'{series:%d-%b-%Y}'

def create_color_col(series):
    if series=='Leave Full Day' or series=='Time In Lieu Full Day' or series=='Bank Holiday':
        hue = '#FA4032'
    elif series=='Leave Half Day' or series=='Time In Lieu Half Day':
        hue = '#FA812F'
    elif series=='Country Support Visit':
        hue='#FCC737'
    elif series=='Conference/Workshop' or series=='Training':
        hue='#FF2929'
    else:
        hue='#A6AEBF'
    
    return hue

# calendar['color'] = calendar.type.apply(create_color_col)
colors = [
    ('Leave Full Day','#FA4032'),('Time In Lieu Full Day','#FA4032'),('Bank Holiday','#FA4032'),
    ('Leave Half Day','#FA812F'),('Time In Lieu Half Day','#FA812F'),
    ('Conference/Workshop','#FF2929'),('Training','#FF2929')
]

hover_data={
    'start_date':False,
    'end_date':False,
    'advisor':False,
    'type':False,
    'from':calendar.start_date.apply(date_prettify),
    'to':calendar.end_date.apply(date_prettify),
    # 'Remarks':calendar.remarks if ~calendar.remarks.isna() else False
    # 'Country':calendar.remarks if calendar[calendar.type=='Country Support Visit'] else False
}

fig = px.timeline(
    data_frame=calendar,
    x_start='start_date',
    x_end='end_date',
    y='advisor',
    color='type',
    hover_name='type',
    hover_data=hover_data,
    text='remarks',
    range_x=[dt.today() - timedelta(15), dt.today() + timedelta(30)],
    labels={'type':''}
)
fig.update_traces(textposition='inside')
fig.update_xaxes(tickformat='%d-%b', dtick=86400000.0*5)

fig.show()

In [28]:
import numpy as np

for year in calendar.start_date.dt.year.unique():
    print(year, np.busday_count(f'{year}-01',f'{year}-12'))

2024 240
2025 238


In [33]:
np.busday_count(calendar.end_date[0], calendar.start_date[0])

TypeError: Iterator operand 0 dtype could not be cast from dtype('<M8[us]') to dtype('<M8[D]') according to the rule 'safe'

In [34]:
calendar.start_date

0    2024-12-18
1    2024-12-25
2    2024-12-30
3    2025-01-01
4    2024-11-01
5    2024-11-07
6    2024-11-01
7    2024-11-22
8    2024-12-25
9    2024-12-30
10   2025-01-01
11   2024-11-01
12   2024-11-06
13   2024-11-13
14   2024-11-28
15   2024-12-25
16   2024-12-30
17   2025-01-01
18   2024-11-21
19   2024-12-12
20   2024-12-25
21   2024-12-30
22   2025-01-01
Name: start_date, dtype: datetime64[ns]